# Notebook to save predictions to 51

## Imports

In [1]:
import fiftyone as fo
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
import cv2
from PIL import Image # TODO to change 
from torchvision import transforms
from torchvision.transforms import (
    ToTensor,
    ToPILImage,
    Resize
)

## Get the dataset and filter it

In [2]:
dataset = fo.load_dataset("esmart_context")

In [3]:
#!fiftyone datasets export -j data/esmart_wip/backups/yyyy_mm_dd.json esmart_wip

In [4]:
train_dataset_51 = dataset.match_tags("TRAIN_RC_AUTO_TIME")
val_dataset_51 = dataset.match_tags("VAL_RC_AUTO_TIME")

In [5]:
fo.launch_app(dataset, port=5151)

Dataset:          esmart_context
Media type:       image
Num samples:      20523
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [8]:
cd ..

/home/selim/Desktop/yolov5_multitask


In [9]:
pwd

'/home/selim/Desktop/yolov5_multitask'

## Run Detect.py on the validation set

In [12]:
 !python detect.py --weights runs/train/exp173/weights/last.pt --source data/multitasks_context.yaml --img-size 640 640 --nosave --save-txt --fiftyone

detect: weights=['runs/train/exp173/weights/last.pt'], source=data/multitasks_context.yaml, data=data/multitasks.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, fiftyone=True
path runs/detect/exp5151
YOLOv5 🚀 v6.1-320-g5eeefe1 Python-3.8.10 torch-1.12.0+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 SUPER, 7960MiB)

Fusing layers... 
YOLOv5s_after_sppf_complex summary: 223 layers, 9529002 parameters, 0 gradients
image 1/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_81036.jpg: 416x640 1 rw_tcd, wet 0.91, Done. (0.303s)
image 2/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_30072.jpg: 448x640 wet 1.0, Done. (0.385s)
image 3/5775 data/multitasks/esmart_

image 61/5775 data/multitasks/esmart_context/20210630_202648_38713.jpg: 480x640 wet 0.91, Done. (0.005s)
image 62/5775 data/multitasks/esmart_context/Log-20220412-192914 Untitled_20649.jpg: 448x640 1 speed_limit, 1 rural-paved, wet 1.0, Done. (0.005s)
image 63/5775 data/multitasks/esmart_context/20210629_174553_20152.jpg: 352x640 1 speed_limit, wet 1.0, Done. (0.238s)
image 64/5775 data/multitasks/esmart_context/Log-20220219-131608 Data Log_34040.jpg: 512x640 1 highway, snowy 1.0, Done. (0.011s)
image 65/5775 data/multitasks/esmart_context/Log-20220222-083904 Data Log_40582.jpg: 448x640 1 rw_tcd, snowy 0.99, Done. (0.006s)
image 66/5775 data/multitasks/esmart_context/20210724_133321_36892.jpg: 480x640 1 rural-paved, wet 0.99, Done. (0.005s)
image 67/5775 data/multitasks/esmart_context/Log-20220227-134325 Data Log_26631.jpg: 448x640 1 highway, wet 0.75, Done. (0.004s)
image 68/5775 data/multitasks/esmart_context/Log-20220126-081950 Data Log_45344.jpg: 416x640 2 speed_limits, 1 urban, we

image 128/5775 data/multitasks/esmart_context/20210630_083253_31808.jpg: 480x640 3 rw_tcds, 1 urban, wet 1.0, Done. (0.004s)
image 129/5775 data/multitasks/esmart_context/Log-20220227-134325 Data Log_22384.jpg: 448x640 1 highway, snowy 0.93, Done. (0.005s)
image 130/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_34301.jpg: 448x640 wet 1.0, Done. (0.004s)
image 131/5775 data/multitasks/esmart_context/Log-20220227-134325 Data Log_28863.jpg: 448x640 1 highway, wet 0.99, Done. (0.004s)
image 132/5775 data/multitasks/esmart_context/20210630_180812_31994.jpg: 480x640 1 highway, dry 1.0, Done. (0.005s)
image 133/5775 data/multitasks/esmart_context/Log-20220219-131608 Data Log_34691.jpg: 512x640 snowy 0.94, Done. (0.005s)
image 134/5775 data/multitasks/esmart_context/20210630_083253_30227.jpg: 480x640 1 rw_tcd, wet 0.98, Done. (0.005s)
image 135/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_93665.jpg: 480x640 3 rw_tcds, 1 highway, wet 0.98, Done. (0.004s)
i

image 194/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_90193.jpg: 480x640 6 rw_tcds, 1 highway, wet 0.58, Done. (0.005s)
image 195/5775 data/multitasks/esmart_context/laval_urban_winter_24616.jpg: 448x640 snowy 1.0, Done. (0.005s)
image 196/5775 data/multitasks/esmart_context/laval_urban_winter_27034.jpg: 448x640 1 speed_limit, 1 urban, wet 0.68, Done. (0.004s)
image 197/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_86163.jpg: 480x640 wet 0.99, Done. (0.005s)
image 198/5775 data/multitasks/esmart_context/Log-20220123-124034 Data Log_41325.jpg: 480x640 1 highway, wet 1.0, Done. (0.004s)
image 199/5775 data/multitasks/esmart_context/20210724_133321_32738.jpg: 480x640 1 rural-paved, dry 0.94, Done. (0.004s)
image 200/5775 data/multitasks/esmart_context/20210629_174553_17827.jpg: 448x640 2 highways, dry 1.0, Done. (0.005s)
image 201/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_48624.jpg: 480x640 1 urban, wet 0.95, Done. (0.005s)
im

image 261/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_41990.jpg: 480x640 1 urban, wet 1.0, Done. (0.005s)
image 262/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_43216.jpg: 480x640 dry 1.0, Done. (0.004s)
image 263/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_98396.jpg: 416x640 wet 0.99, Done. (0.005s)
image 264/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_48858.jpg: 480x640 1 rw_tcd, 1 highway, dry 0.94, Done. (0.005s)
image 265/5775 data/multitasks/esmart_context/Log-20220412-192914 Untitled_20246.jpg: 448x640 1 highway, wet 0.99, Done. (0.005s)
image 266/5775 data/multitasks/esmart_context/Log-20220324-064020_87520.jpg: 448x640 3 rw_tcds, wet 1.0, Done. (0.005s)
image 267/5775 data/multitasks/esmart_context/20210724_133321_32521.jpg: 480x640 1 highway, dry 0.97, Done. (0.005s)
image 268/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_28615.jpg: 448x640 wet 1.0, Done. (0.005s)
image 269/5

...

image 5481/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_32955.jpg: 448x640 wet 1.0, Done. (0.005s)
image 5482/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_90317.jpg: 480x640 5 rw_tcds, 1 highway, dry 0.82, Done. (0.005s)
image 5483/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_54004.jpg: 480x640 1 highway, dry 0.99, Done. (0.004s)
image 5484/5775 data/multitasks/esmart_context/20210630_202648_34919.jpg: 480x640 snowy 0.67, Done. (0.004s)
image 5485/5775 data/multitasks/esmart_context/Log-20220413-104057 Untitled_22663.jpg: 448x640 1 urban, wet 1.0, Done. (0.005s)
image 5486/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_48841.jpg: 480x640 snowy 0.98, Done. (0.005s)
image 5487/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_52392.jpg: 480x640 2 highways, wet 0.96, Done. (0.004s)
image 5488/5775 data/multitasks/esmart_context/20210630_180812_31901.jpg: 480x640 1 highway, dry 1.0, Done. (0.004s)


image 5546/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_78866.jpg: 416x640 1 speed_limit, wet 1.0, Done. (0.005s)
image 5547/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_30103.jpg: 448x640 wet 1.0, Done. (0.005s)
image 5548/5775 data/multitasks/esmart_context/20210630_180812_33606.jpg: 480x640 dry 1.0, Done. (0.005s)
image 5549/5775 data/multitasks/esmart_context/20210630_083253_35373.jpg: 480x640 1 speed_limit, dry 0.79, Done. (0.004s)
image 5550/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_34301.jpg: 448x640 wet 1.0, Done. (0.005s)
image 5551/5775 data/multitasks/esmart_context/Log-20220219-161929 Data Log_12433.jpg: 352x640 1 highway, wet 1.0, Done. (0.005s)
image 5552/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_91929.jpg: 480x640 dry 1.0, Done. (0.005s)
image 5553/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_54948.jpg: 480x640 1 urban, wet 1.0, Done. (0.004s)
image 5554/5775 data/mu

image 5614/5775 data/multitasks/esmart_context/20210630_180812_37039.jpg: 480x640 1 highway, dry 0.96, Done. (0.005s)
image 5615/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_25236.jpg: 448x640 1 rural-paved, wet 1.0, Done. (0.005s)
image 5616/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_44270.jpg: 480x640 1 highway, dry 1.0, Done. (0.005s)
image 5617/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_52375.jpg: 480x640 1 urban, wet 1.0, Done. (0.004s)
image 5618/5775 data/multitasks/esmart_context/Log-20220123-124034 Data Log_42513.jpg: 480x640 wet 1.0, Done. (0.004s)
image 5619/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_52547.jpg: 480x640 1 highway, dry 0.63, Done. (0.004s)
image 5620/5775 data/multitasks/esmart_context/20210630_202648_31075.jpg: 480x640 1 speed_limit, 1 highway, dry 0.6, Done. (0.004s)
image 5621/5775 data/multitasks/esmart_context/Log-20220419-101657 Data Log_32025.jpg: 448x640 1 urban, wet 

image 5681/5775 data/multitasks/esmart_context/20210724_133321_42782.jpg: 480x640 1 rw_tcd, 1 urban, wet 0.99, Done. (0.005s)
image 5682/5775 data/multitasks/esmart_context/Log-20220219-131608 Data Log_32118.jpg: 512x640 1 speed_limit, snowy 1.0, Done. (0.005s)
image 5683/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_54049.jpg: 480x640 1 urban, wet 1.0, Done. (0.005s)
image 5684/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_56343.jpg: 480x640 wet 0.91, Done. (0.004s)
image 5685/5775 data/multitasks/esmart_context/2021_07_29_12_00_52.20210728_072638_52764.jpg: 416x640 3 rw_tcds, 1 highway, wet 0.5, Done. (0.005s)
image 5686/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_98210.jpg: 416x640 1 speed_limit, 1 urban, wet 1.0, Done. (0.004s)
image 5687/5775 data/multitasks/esmart_context/20210630_083253_35621.jpg: 480x640 8 rw_tcds, 1 highway, dry 1.0, Done. (0.005s)
image 5688/5775 data/multitasks/esmart_context/20210724_133321_35187.jp

image 5747/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_78835.jpg: 416x640 1 speed_limit, 1 highway, wet 1.0, Done. (0.004s)
image 5748/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_56932.jpg: 480x640 1 speed_limit, wet 1.0, Done. (0.005s)
image 5749/5775 data/multitasks/esmart_context/Log-20220123-124034 Data Log_37791.jpg: 480x640 wet 1.0, Done. (0.004s)
image 5750/5775 data/multitasks/esmart_context/Log-20220419-063722 Data Log_45741.jpg: 480x640 wet 0.97, Done. (0.004s)
image 5751/5775 data/multitasks/esmart_context/Log-20220414-064154 Data Log_49571.jpg: 480x640 1 rw_tcd, 1 highway, dry 0.91, Done. (0.004s)
image 5752/5775 data/multitasks/esmart_context/Log-20220219-131608 Data Log_35373.jpg: 512x640 snowy 1.0, Done. (0.005s)
image 5753/5775 data/multitasks/esmart_context/Log-20220426-064124 Data Log_97571.jpg: 480x640 wet 0.68, Done. (0.005s)
image 5754/5775 data/multitasks/esmart_context/Log-20220413-104057 Untitled_20741.jpg: 448x640 1 urb

## Delete old predictions for road_condtions in order to save the new ones

In [13]:
# a = 0
# with fo.ProgressBar() as pb:
#     for sample in pb(val_dataset_51):
#     #     print(sample)
#         try:
#             del sample["classif_road_cond_yolo"]
#             sample.save()
#         except:
#             a += 1

 100% |███████████████| 5775/5775 [3.9s elapsed, 0s remaining, 2.0K samples/s]        


## Save predictions to Fiftyone

In [19]:
def get_prediction(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        condition, confidence = lines[0].split(',')[0], lines[0].split(',')[1]
        print(lines)
    return condition, confidence

In [22]:
# directory = '/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls'
# with fo.ProgressBar() as pb:
#     for sample in pb(val_dataset_51):
#         try:
#             img = sample.filepath.split('/')[-1]
#             if img.endswith('.jpg'):
#                 img_name = img.replace('jpg','txt')
#             elif img.endswith('.png'):
#                 img_name = img.replace('png','txt')
#             path = f"{directory}/{img_name}"
#             pred, conf = get_prediction(path)
#             sample["classif_road_cond_yolo"] = fo.Classifications(classifications=[fo.Classification(label=pred, confidence=conf)])
#             sample.save()
#         except:
#             print(img_name)
#             break

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_19656.txt
['wet,0.57']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5eae51c7c20d3a37bcd',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '0.57',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_19687.txt
['wet,0.99']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5eae51c7c20d3a37bd2',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '0.99',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_19718.txt
['wet,0.92']
<Classifications: {
    'classifications': BaseList(

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_20400.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ebe51c7c20d3a37c45',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_20431.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ebe51c7c20d3a37c4a',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_20462.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
   

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21051.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ebe51c7c20d3a37cae',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21082.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ebe51c7c20d3a37cb3',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21113.txt
['wet,0.99']
<Classifications: {
    'classifications': BaseList([
  

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21671.txt
['wet,0.98']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ece51c7c20d3a37d12',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '0.98',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21702.txt
['wet,0.99']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ece51c7c20d3a37d17',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '0.99',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_21733.txt
['wet,0.57']
<Classifications: {
    'classifications': BaseList(

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_22291.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ece51c7c20d3a37d76',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_22322.txt
['dry,0.85']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe5ece51c7c20d3a37d7b',
            'tags': BaseList([]),
            'label': 'dry',
            'confidence': '0.85',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/Log-20220412-190445 Untitled_22353.txt
['wet,1.0']
<Classifications: {
    'classifications': BaseList([
 

...

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_24709.txt
['wet,0.5']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66be51c7c20d3a3ea6d',
            'tags': BaseList([]),
            'label': 'wet',
            'confidence': '0.5',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_24740.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66be51c7c20d3a3ea72',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_24771.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {


/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28150.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66be51c7c20d3a3eae5',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28181.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66be51c7c20d3a3eaea',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28212.txt
['snowy,0.65']
<Classifications: {
    'classifications': BaseList([
        <Classificatio

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28863.txt
['snowy,0.62']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66be51c7c20d3a3eb58',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '0.62',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28894.txt
['snowy,0.7']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66ce51c7c20d3a3eb5d',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '0.7',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_28925.txt
['snowy,0.99']
<Classifications: {
    'classifications': BaseList([
        <Classificat

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_32521.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66ce51c7c20d3a3ebc6',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_32552.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66ce51c7c20d3a3ebcb',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_32583.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification

/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_34536.txt
['snowy,1.0']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66ce51c7c20d3a3ec39',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '1.0',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_34567.txt
['snowy,0.99']
<Classifications: {
    'classifications': BaseList([
        <Classification: {
            'id': '62ebe66ce51c7c20d3a3ec3e',
            'tags': BaseList([]),
            'label': 'snowy',
            'confidence': '0.99',
            'logits': None,
        }>,
    ]),
    'logits': None,
}>
/home/selim/Desktop/yolov5_multitask/runs/detect/exp5151/labels_cls/laval_urban_winter_34598.txt
['snowy,0.99']
<Classifications: {
    'classifications': BaseList([
        <Classificat

## Check the ambiguous images

In [27]:
for sample in dataset:
#     print(sample.tags)
    if 'ambiguous_road_cond' in sample.tags:
        print(sample.tags)
        break

['COCO', 'TRAIN_ROAD_COND', 'TRAINING', 'ambiguous_road_cond', 'TRAIN_RC_AUTO_TIME', 'TRAIN_RC_RND_FR']


In [2]:
import torch

## Check captum

In [4]:
test = torch.tensor([0.2,0.5,4])
torch.topk(test, 1)

torch.return_types.topk(
values=tensor([4.]),
indices=tensor([2]))